In [3]:
import warnings
import requests
import json
import os
from jsonpath_ng import jsonpath, parse
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')

def createJson(Output_response, file_path): #file_path="data/data.json"
    jsonString = json.dumps(Output_response, indent=4)
    jsonFile = open(file_path, "w")
    jsonFile.write(jsonString)
    jsonFile.close()

def create_Csv_for_required_data(jsonPath,JsonInput,Csv_filepath): #jsonPath = '$[*].modules[*].questions'
    jsonpath_expression = parse(jsonPath)
    Questions = jsonpath_expression.find(JsonInput)
    df_all=pd.DataFrame()
    ArrLength = len(Questions)
    jsonvalue = []
    for i in range(ArrLength):
        jsonvalue= json.dumps(Questions[i].value,indent=4)
        df= pd.read_json(jsonvalue)
        df_all= pd.concat([df_all, df])

    df_all = df_all.reset_index()
    df_all = df_all.loc[:, ["groupName","variableName","versionNumber"]]
    df_all = df_all.dropna()
    return df_all


def get_data(api, header,fileName):
    response = requests.get(f"{api}",headers=header,verify = False)
    if response.status_code == 200:
        print(f"{fileName}-Done")
        df_all_local2 = pd.DataFrame()
        Output_response = response.json()
        createJson(Output_response,fileName)
        jsonPath = "$[*].modules[*].questions" #?(@.type!='presentation')
        df_all_local2 = create_Csv_for_required_data(jsonPath,Output_response,"data/test.csv")
        return df_all_local2
    else:
        print(f"Error {response.status_code} for {fileName}")

print("input Base URL")
BaseUrl = input() 

print("input Bearer Token to authenticate: ")
bearerToken = input()        
my_headers = {'Authorization' : bearerToken}

products = ["Packevaluatescreening-6","Linkplustv-13"]
df_all_final=pd.DataFrame()
df_all_local=pd.DataFrame()
for product in products:
    Prod_data = product.split("-")
    ProductId = Prod_data[0]
    versionId = Prod_data[1]
    URL = f"{BaseUrl}/{ProductId}/stage/production/configuration?version={versionId}"
    fileName = f"data/{ProductId}_{versionId}_Produc_Template.json"
    df_all_local = get_data(URL,my_headers,fileName)
    df_all_final = pd.concat([df_all_final,df_all_local])

df_all_final = df_all_final.drop_duplicates()
df_all_final.index = np.arange(1, len(df_all_final) + 1)
df_all_final = df_all_final.reset_index(names="S.No")
open("data/test.csv", "a")
df_all_final.to_csv("data/test.csv",index=False)




input Base URL
https://prod-tesseract-ne-pre-prod.azurewebsites.net/ui/product
input Bearer Token to authenticate: 
Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJmM2Q1ZWI2MC1iNTEyLTQ1YWMtODI0YS1iMDQ4MDRjZDFmZmEiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8xZTM1NWMwNC1lMGE0LTQyZWQtOGUyZC03MzUxNTkxZjBlZjEvIiwiaWF0IjoxNjgzMzQ3NjkxLCJuYmYiOjE2ODMzNDc2OTEsImV4cCI6MTY4MzM1MTc4NSwiYWNyIjoiMSIsImFpbyI6IkFWUUFxLzhUQUFBQUN3QTdNOXR5dXhuVDZ2dVNFZWlkZjY3a0VDZnN0SWNXb3pQby9uMy9tcC9kVEtUbnBmSU5uMVd2T0JXYnk4dk1tRS9jYW5VQ1E1d3Jpc3RaZytMMEcrajlWSktKY1A4ZjdQU0VIa2dPRE53PSIsImFtciI6WyJwd2QiLCJtZmEiXSwiYXBwaWQiOiIwOTE1ZjM5YS1lMDVkLTRhOTAtOTY4ZC1iMmExNDZkYjQzZTYiLCJhcHBpZGFjciI6IjAiLCJmYW1pbHlfbmFtZSI6IkthbW1hcmEiLCJnaXZlbl9uYW1lIjoiUmF2aSIsImlwYWRkciI6IjEzNC4yMzguMjM4LjY3IiwibmFtZSI6IlJhdmkgS2FtbWFyYSIsIm9pZCI6ImY2OTY3YTY3LTQ3YmQtNDdmMy1hYmVjLWE4YmIzOWJkNzIyNSIsIm9ucHJlbV9zaWQiOiJTLTEtNS0yMS0zNDMyMjIxNDA5